# Hw1 Documentation

In [1]:
from default import *

## Pseudo code of iterative segmenter

    while heap is nonempty:
        entry = top entry in the heap

        if chart[endindex] has a previous entry, preventry
            if entry has a higher probability than preventry:
                chart[endindex] = entry
            if entry has a lower or equal probability than preventry:
                continue
        else
            chart[endindex] = entry

        for each newword that matches input starting at position endindex+1
            newentry = Entry(newword, endindex+1, entry.log-probability + logPw(newword), entry)
            if newentry does not exist in heap:
                insert newentry into heap

    finalindex is the length of input
    finalentry = chart[finalindex]
    The best segmentation starts from finalentry and follows the back-pointer recursively until the first word

## Actual implementation of iterative segmenter

In [2]:
   def segment(self, text):
        "Return a list of words that is the best segmentation of text."
        if not text: return []

        ## Initialize the heap ##  
        heap = []
        splitted_text = self.splits(text)

        for w,rem in splitted_text:
            #  Entry(word, start-position, log-probability, back-pointer)
            entry = (w, 0, math.log10(self.Pw(w)), None)
            heap.append(entry)

        heap.sort(key=takeProb)

        # # initialize chart: the dynamic programming table to store the argmax for every prefix of input
        # # indexed by character position in input
        chart = [None for i in range(len(text))]

        ## Iteratively fill in chart[i] for all i ##
        while heap:
            entry = heap.pop()
            entry_word = entry[0]
            entry_start = entry[1]
            entry_prob = entry[2]
            entry_backptr = entry[3]

            # get the endindex based on the length of the word in entry
            endindex = entry_start + len(entry_word)  - 1 

            if chart[endindex]:
                preventry = chart[endindex]
                preventry_word = preventry[0]
                preventry_start = preventry[1]
                preventry_prob = preventry[2]
                preventry_backptr = preventry[3]

                if entry_prob > preventry_prob:
                    # if entry has a higher probability than preventry
                    chart[endindex] = entry
                if entry_prob <= preventry_prob:
                     ## we have already found a good segmentation until endindex ##
                    continue
            else:
                chart[endindex] = entry
            
            splitted_text = self.splits(text[(endindex + 1):])

            for newword,rem in splitted_text:
                newentry = (newword, endindex + 1, entry_prob + math.log10(self.Pw(newword)), entry)
                if newentry not in heap:
                    heap.append(newentry)

            heap.sort(key=takeProb)

        # ## Get the best segmentation ##
        finalindex = len(text)
        finalentry = chart[finalindex - 1]
        segmentation = self.recursive_back(finalentry)

        return segmentation

## Run the unigram solution on dev following the iterative segmenter

In [3]:
Pw = Pdist(data=datafile("data/count_1w.txt"),N = None, missingfn = smooth)
segmenter = Segment(Pw) # note that the default solution for this homework ignores the unigram counts
output_full = []
with open("data/input/dev.txt") as f:
    for line in f:
        output = segmenter.segment(line.strip())
        output_full.append(output)
print("\n".join(output_full[:3])) # print out the first three lines of output as a sanity check

中 美 在 沪 签订 高 科技 合作 协议 
新华社 上海 八月 三十一日 电 （ 记者 白国良 、 夏儒阁 ） 
“ 中 美 合作 高 科技 项目 签字 仪式 ” 今天 在 上海 举行 。 


## Add smooth function to give probability for unknown words

In [4]:
def smooth(word, N):
    return 10000./(N * 1000**len(word))

## Actual implementation of iterative segmenter for bigram

In [5]:
def segment(self, text, prev = '<S>'):
        if (text, prev) in self.table:
            return self.table[(text, prev)]

        if not text: return 0.0, []

        segmentation = []
        
        splitted_text = self.splits(text)

        for first, rem in splitted_text:

            # recursively call segment
            restProb, rest = self.segment(rem,first)

            prob = log10(self.cPw(prev,first))
            segmentation.append((prob + restProb, [first] + rest))

        finalentry = segmentation[0]
        for index in range(len(segmentation)):
            if segmentation[index][0] > finalentry[0]:
                finalentry = segmentation[index]

        self.table[text, prev] = finalentry

        return finalentry

## Evaluate the iterative segmenter output

In [6]:
from zhsegment_check import fscore
with open('data/reference/dev.out', 'r') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
    tally = fscore(ref_data, output_full)
    print("score: {:.2f}".format(tally), file=sys.stderr)


score: 0.93


## Analysis
We tried to use unigram and bigram. The algorithm of unigrame is iterative segmenter which is similar to the algorithm on homework page. We designed another alogprithm for bigram which made the result better. A variable L in function splits is used to limit the maximum length of a word. According to our tests, we chose 5 to be the maximun length.  We also implemented a smooth function to give unknown words which does not appeared in corpus a probability of 10000./(N * 1000**len(word)) in order to avoid zero probability. Each unknown words will be punished harder compared to previous homework since Chinese words are characters which do not have long length.